In [5]:
import time, threading, queue, sys, gc
from typing import Tuple

import numpy as np
import pyqtgraph as pg
from PyQt6 import QtCore

In [6]:
# ------------------------------------------------------------  parameters
DEBUG           = True           # set True for jitter log
SAMPLE_RATE     = 100_000         # Hz
CHUNK_SEC       = 0.2             # s
N_SAMPLES       = int(SAMPLE_RATE * CHUNK_SEC)
QUEUE_DEPTH     = 4               # raw AB backlog  (≈0.8 s)
QUAD_DEPTH      = 4               # processed backlog (same)
RUN_SEC         = 10              # auto‑stop after 10 s
DISPLAY_SEC       = 0.02            # *** fixed x-axis window width (s) ***
GUI_INTERVAL_MS   = 50             # *** refresh every 50 ms (≈20 FPS) ***

PULSE_HEIGHT    = 5.0             # amplitude
PULSE_WIDTH     = 0.002           # period  (s)
PULSE_DUTY      = 0.5             # duty
PULSE_PHASE_A   = 0.0             # phase offset (s)
PULSE_PHASE_B   = 0.0005          # phase offset (s)

QUADPULSE_WIDTH = 0.00025         # width (s)
THRESHOLD_DEFAULT = 2.5           # logic threshold (V)



# ------------------------------------------------------------  queues & stop flag
buf_q  = queue.Queue(maxsize=QUEUE_DEPTH)   # raw (t, A, B)
quad_q = queue.Queue(maxsize=QUAD_DEPTH)    # processed (t, A, B, quad)
stop_ev = threading.Event()

In [ ]:
def gen_chunk_pulse(t: np.ndarray, *, height: float = PULSE_HEIGHT,
                    width: float = PULSE_WIDTH, duty: float = PULSE_DUTY,
                    phase: float = 0.0) -> np.ndarray:
    mod = (t + phase) % width
    return np.where(mod < duty * width, height, 0.0).astype(np.float32)

REL_AXIS = np.arange(N_SAMPLES, dtype=np.float32) / SAMPLE_RATE  # 0 … 0.199 999 s

# ------------------------------------------------------------  AB → direction → quad helpers

def gen_pulse_direction(dA: np.ndarray, dB: np.ndarray, *, threshold: float) -> np.ndarray:
    A = dA > threshold
    B = dB > threshold
    A_prev = np.concatenate(([A[0]], A[:-1]))
    B_prev = np.concatenate(([B[0]], B[:-1]))
    return ((B_prev ^ A).astype(int) - (A_prev ^ B).astype(int)).astype(np.int8)

def pulse_count(dir_log: np.ndarray) -> int:
    return int(np.sum(dir_log))

def gen_quad_pulse(t: np.ndarray, dir_log: np.ndarray, width: float,
                   height: float, sampling_rate: int) -> np.ndarray:
    samples = int(width * sampling_rate)
    if samples <= 0:
        return np.zeros_like(dir_log, dtype=np.float32)
    base = np.full(samples, height, dtype=np.float32)
    return np.convolve(dir_log, base, mode="full")[:len(t)]

# ------------------------------------------------------------  producer thread

def generator() -> None:
    """Generate AB rectangular‑wave chunks at real‑time cadence.
    Drops the newest chunk if raw queue is already full (latency > 0.8 s).
    """
    chunk_idx = 0
    next_t = time.perf_counter()
    while not stop_ev.is_set():
        base   = chunk_idx * CHUNK_SEC
        t_axis = REL_AXIS + base
        pulse_A = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_A)
        pulse_B = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_B)

        try:
            buf_q.put_nowait((t_axis, pulse_A, pulse_B))
        except queue.Full:
            pass  # drop chunk

        chunk_idx += 1
        next_t += CHUNK_SEC
        sleep = next_t - time.perf_counter()
        if sleep > 0:
            time.sleep(sleep)
        else:
            next_t = time.perf_counter()

# ------------------------------------------------------------  consumer thread

def process_chunk(t: np.ndarray, dA: np.ndarray, dB: np.ndarray) -> Tuple[int, np.ndarray]:
    dir_log = gen_pulse_direction(dA, dB, threshold=THRESHOLD_DEFAULT)
    return pulse_count(dir_log), gen_quad_pulse(t, dir_log, QUADPULSE_WIDTH, PULSE_HEIGHT, SAMPLE_RATE)

def consumer() -> None:
    last_ts = time.perf_counter()
    while not stop_ev.is_set():
        try:
            t, pulse_A, pulse_B = buf_q.get(timeout=0.5)
        except queue.Empty:
            continue
        now = time.perf_counter()
        count, quad_sig = process_chunk(t, pulse_A, pulse_B)

        # debug print ---------------------------------------------------
        if DEBUG:
            jitter_ms = (now - last_ts) * 1e3
            print(
                f"UNIX EPOCH = {now:.6f}  iter = {jitter_ms:6.2f} ms  "
                f"t = {t[0]:.3f} s ~ {t[-1]:.3f} s  count = {count:+d}")
        last_ts = now

        # send downstream ----------------------------------------------
        try:
            quad_q.put_nowait((t, pulse_A, pulse_B, quad_sig))
        except queue.Full:
            pass  # drop

        # explicit GC of large arrays ----------------------------------
        del t, pulse_A, pulse_B, quad_sig
        gc.collect()
        buf_q.task_done()

# ------------------------------------------------------------  PyQtGraph GUI
HISTORY = int(SAMPLE_RATE * DISPLAY_SEC) # x-axis history (samples)

def start_gui() -> None:
    pg.setConfigOptions(useOpenGL=True, background='w', foreground='k')  # optional: GPU accel
    app = pg.mkQApp("Graph plot")

    win = pg.GraphicsLayoutWidget(title="Graph plot")
    win.show()

    # top plot: A/B
    plt_ab  = win.addPlot(row=0, col=0, title="Phase A / Phase B")
    curve_A = plt_ab.plot(pen = pg.mkPen(color='#ff4b00', width=3))
    curve_B = plt_ab.plot(pen = pg.mkPen(color='#005aff', width=3))
    plt_ab.setLabel(axis='left', text='Amplitude (V)')
    plt_ab.setLabel(axis='bottom', text='Time [s]')
    plt_ab.setYRange(-0.5, PULSE_HEIGHT + 0.5)
    plt_ab.showGrid(x=True, y=True, alpha=0.3)

    # bottom plot: quad
    plt_q  = win.addPlot(row=1, col=0, title="Quad signal")
    curve_Q = plt_q.plot(pen = pg.mkPen(color='#03af7a', width=3))
    plt_q.setYRange(-0.5 - PULSE_HEIGHT, PULSE_HEIGHT + 0.5)
    plt_q.setLabel(axis='left', text='Amplitude (V)')
    plt_q.setLabel(axis='bottom', text='Time [s]')
    plt_q.showGrid(x=True, y=True, alpha=0.3)

    xs = ya = yb = yq = np.empty(0, dtype=np.float32)

    def refresh():
        nonlocal xs, ya, yb, yq
        try:
            while True:
                t_ax, pA, pB, qsig = quad_q.get_nowait()
                xs = np.concatenate((xs, t_ax))[-HISTORY:]
                ya = np.concatenate((ya, pA))[-HISTORY:]
                yb = np.concatenate((yb, pB))[-HISTORY:]
                yq = np.concatenate((yq, qsig))[-HISTORY:]
                quad_q.task_done()
        except queue.Empty:
            pass

        # fixed x‑axis window
        if xs.size:
            start = xs[-1] - DISPLAY_SEC
            plt_ab.setXRange(start, xs[-1], padding=0)
            plt_q.setXRange(start, xs[-1], padding=0)

        curve_A.setData(xs, ya)
        curve_B.setData(xs, yb)
        curve_Q.setData(xs, yq)

    timer = QtCore.QTimer()
    timer.timeout.connect(refresh)
    timer.start(GUI_INTERVAL_MS)  # ≈20 FPS

    # auto‑stop after RUN_SEC
    QtCore.QTimer.singleShot(RUN_SEC * 1000, lambda: (stop_ev.set(), app.quit()))
    app.exec()


In [8]:
gen_th = threading.Thread(target=generator, daemon=True)
con_th = threading.Thread(target=consumer, daemon=True)

gen_th.start()
con_th.start()

start_gui()

# join threads and exit
stop_ev.set()
gen_th.join()
con_th.join()

print("Graceful shutdown.")


UNIX EPOCH = 187041.576395  iter = 283.59 ms  t = 0.000 s ~ 0.200 s  count = -399
UNIX EPOCH = 187041.671111  iter =  94.72 ms  t = 0.200 s ~ 0.400 s  count = -399
UNIX EPOCH = 187041.720608  iter =  49.50 ms  t = 0.400 s ~ 0.600 s  count = -399
UNIX EPOCH = 187041.906712  iter = 186.10 ms  t = 0.600 s ~ 0.800 s  count = -399
UNIX EPOCH = 187042.135291  iter = 228.58 ms  t = 0.800 s ~ 1.000 s  count = -399
UNIX EPOCH = 187042.318412  iter = 183.12 ms  t = 1.000 s ~ 1.200 s  count = -400
UNIX EPOCH = 187042.505538  iter = 187.13 ms  t = 1.200 s ~ 1.400 s  count = -399
UNIX EPOCH = 187042.710365  iter = 204.83 ms  t = 1.400 s ~ 1.600 s  count = -399
UNIX EPOCH = 187042.905343  iter = 194.98 ms  t = 1.600 s ~ 1.800 s  count = -399
UNIX EPOCH = 187043.105372  iter = 200.03 ms  t = 1.800 s ~ 2.000 s  count = -399
UNIX EPOCH = 187043.312678  iter = 207.31 ms  t = 2.000 s ~ 2.200 s  count = -400
UNIX EPOCH = 187043.504387  iter = 191.71 ms  t = 2.200 s ~ 2.400 s  count = -399
UNIX EPOCH = 187